### Notebook to adjust PetroNER for Entity Linking task

In [176]:
from owlready2 import *
import gensim
from conllu import parse_incr, parse
import numpy as np

Load ontology and embeddings

In [217]:
#OntoGeoLógica populada (OntoGeoLogicaPopulada.owl)
onto = get_ontology("../../KnowledgeGraph/OntoGeoLogicaPovoadaInstanciasRelacoes.owl")
onto.load()

get_ontology("http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#")

In [218]:
# Modelo OWL2Vec - 
PetroOntoVec = gensim.models.Word2Vec.load("../../Embeddings/PetroOntoVec/PetroOntoVec_simples/outputontology.embeddings")

Load PetroNER (file in CONLLU format)

In [219]:
# CONLLU parser
boleltins = open("../PetroNER/petroner-2022-12-19.conllu", "r", encoding="utf-8")
sentences = parse_incr(boleltins)

### Preparing and processing the dataset

Função que recebe o texto anotado em formato CONLLU e processa para o treinamento. Tem como saída as sentenças acrescida das tags para indicar as entidades, bem como com a entidade no início do texto; lista com as entidades (instâncias extraídas do texto); e lista com as classes que as entidades pertencem. O texto sai com o seguinte formato:  
"[Nome da classe] Nome da entidade | blablabla blablabla blablabla [E] Nome da entidade [/E] blablabla blablabla." 

In [220]:
# Função que recebe
def preparar_dataset(tokenlist):

    #sent = TokenList([])
    sent_orig = ''   # String para compor a sentença original
    sents_NER = []   # Lista para armazenar as sentenças com indicação das entidades
    NER_on = False   # Variável para indicar se está iterando por token de entidades
    NER_tok = ''     # string para armazenar os tokens das entidades que estão sendo iteradas
    MW_on = False    # Variável para indicar se está iterando por tolkens multiword
    MW_tok = ''      # string para armazenar o termo multiword
    MW_end = 0       # ID final do token multiword
    ENT = []
    ENT_class = []
    ENT_ID = []

    for tok in tokenlist:
        # ignorando id não inteiro (tuplas), ou seja token multiword
        if type(tok['id']) == int:

            # Verifica se o token inicia uma entidade ("B")
            if tok['deps'][0] == 'B':
                
                # Verifica se a variável NER_on está ativado (No caso de um tok "B" seguido de outro)
                if NER_on == True:
                    
                    NER_on = False  # Desativa a variável NER_on

                    # Para sentença mais recente da lista sents_NER, inclui a entidade no início e o marcador [/E] no final 
                    sents_NER[-1] = '[' + ENT_class[-1] + '] ' + NER_tok + '| ' + sents_NER[-1].strip() + ' [/E]'

                    # Incluir o NER_tok na lista ENT
                    ENT.append(NER_tok.strip())

                    NER_tok = '' # Apaga a string NER_tok

                NER_on = True  # Ativa a variável NER_on
                # Acrescenta uma sentença com o marcador [E] na listas de sentenças com NER
                sents_NER.append(sent_orig + ' [E]')
                # Incluir classe na lista ENT_class
                ENT_class.append(tok['deps'][2:])
                try:
                    ENT_ID.append(tok['misc']['grafo'])
                except:
                    ENT_ID.append(None)

            # Identifica o primeiro token "O" após uma sequência de entidades
            if (tok['deps'][0] == 'O') & NER_on:
                
                NER_on = False  # Desativa a variável NER_on

                # Para sentença mais recente da lista sents_NER, 
                # inclui a classe e a entidade no início e o marcador [/E] no final 
                sents_NER[-1] = '[' + ENT_class[-1] + '] ' + NER_tok + '| ' + sents_NER[-1].strip() + ' [/E]'

                # Incluir o NER_tok na lista ENT
                ENT.append(NER_tok.strip())

                NER_tok = '' # Apaga a string NER_tok
                
            # Identifica se o último token da sequência tem o marcador "B" ou "I".    
            if (tokenlist[-1]['id'] == tok['id']) & ((tok['deps'][0] == 'B') | (tok['deps'][0] == 'I')):
                NER_on = False  # Desativa a variável NER_on

                # Para sentença mais recente da lista sents_NER, 
                # inclui a classe e a entidade no início e o marcador [/E] no final 
                sents_NER[-1] = '[' + ENT_class[-1] + '] ' + NER_tok + '| ' + sents_NER[-1].strip() + ' [/E]'

                # Incluir o NER_tok na lista ENT
                ENT.append(NER_tok.strip())

                NER_tok = '' # Apaga a string NER_tok

            # Verifica se está iterando por uma multiword ou não
            if not MW_on:

                # Acrecenta o token à string da entidade
                if NER_on:
                    NER_tok = NER_tok + tok['form'] + ' '

                # Se o token for uma pontuação, não inclui espaço entre o token anterior 
                if tok['upos'] == 'PUNCT':
                    # O token é incluído na senteça original e em todas as sentenças da lista sents_NER
                    sent_orig = sent_orig + tok['form']
                    for i in range(len(sents_NER)):
                        sents_NER[i] = sents_NER[i] + tok['form']
                else:
                    sent_orig = sent_orig + ' ' + tok['form']
                    for i in range(len(sents_NER)):
                        sents_NER[i] = sents_NER[i] + ' ' + tok['form']

            else:
                # Se o token for o último termo da multiword, desativamos a variável MW_on
                if tok['id'] == MW_end:
                    MW_on = False

                    # Acrecenta a multiword à string da entidade
                    if NER_on:
                        NER_tok = NER_tok + MW_tok + ' '

                    #Ao invés de incluir nas senteças o token, vamos incluir o MW_tok 
                    # Se o token for uma pontuação, não inclui espaço entre o token anterior 
                    if tok['upos'] == 'PUNCT':
                        # O token é incluído na senteça original e em todas as sentenças da lista sents_NER
                        sent_orig = sent_orig + MW_tok
                        for i in range(len(sents_NER)):
                            sents_NER[i] = sents_NER[i] + MW_tok
                    else:
                        sent_orig = sent_orig + ' ' + MW_tok
                        for i in range(len(sents_NER)):
                            sents_NER[i] = sents_NER[i] + ' ' + MW_tok

        # Se a ID do token for referente a uma multiword, ativamos a variável MW_on, 
        # armazenamos o termo MW_tok e o valor da 'id' final
        else:
            MW_on = True
            MW_tok = tok['form']
            MW_end = tok['id'][-1]
    
    return(sents_NER, ENT, ENT_class, ENT_ID)

In [221]:
'''
for tokenlist in sentences:
    for tok in tokenlist:
        try:
            if tok['deps'][0] == 'B':
                #print(tok['misc'])
                print(tok['misc']['grafo'])
        except:
            pass
    break  
    
'''

"\nfor tokenlist in sentences:\n    for tok in tokenlist:\n        try:\n            if tok['deps'][0] == 'B':\n                #print(tok['misc'])\n                print(tok['misc']['grafo'])\n        except:\n            pass\n    break  \n    \n"

In [222]:
# Separando as listas de sentenças, entidades e classes
text = []
entities = []
classes_NER = [] 
ENT_ID = []

# Iterando por todas as sentenças em formato CONLLU para preparar o dataset
n = 0
for tokenlist in sentences:
    n = n + 1

    text_n, entities_n, classes_NER_n, ENT_ID_n = preparar_dataset(tokenlist)
    
    text = text + text_n
    entities = entities + entities_n
    classes_NER = classes_NER + classes_NER_n
    ENT_ID = ENT_ID + ENT_ID_n
    
print('Total de sentenças anotadas: ', n)
print('N° de sentenças: ', len(text))
print ('N° de entidades: ', len(entities))
print('N° de classes: ', len(classes_NER))
print('N° de classes: ', len(ENT_ID))

Total de sentenças anotadas:  24035
N° de sentenças:  18820
N° de entidades:  18820
N° de classes:  18820
N° de classes:  18820


In [223]:
count = 0
for i in ENT_ID:
    if i == None:
        count = count + 1
        
print('Número de entidades sem ID: ', count)

Número de entidades sem ID:  9172


### Buscando vetores do PetroOntoVec para cada ID

In [224]:
text_new = []
entities_new = []
classes_NER_new = [] 
ENT_ID_new = []
URI_Vec_new = []

n = 0 
#m = 0
#Iterando por todas as sentenças, desconsiderando aquelas cujas entidades não possuem ENT_ID
for i in range(len(text)):
    n = n + 1
    if ENT_ID[i] != None:
        # Quebrando as entidades que possuem mais de um ENT_ID (ex: "#Aptian,#Miocene" e "#Cenozoic,#Mesozoic")
        ENT_ID_n = ENT_ID[i].split(',#')
        
        # Iterando pelas ENT_ID após serem quebradas e acrescentando '#" naquelas que perderam na operação de split 
        for ent in ENT_ID_n:
            if ent[0] != '#':
                ent = '#' + ent
            try:
                #Acrescentar o ID da entidade e o vetor da URI
                URI_Vec_new.append(PetroOntoVec['http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2' + ent])
                ENT_ID_new.append(ent)
               
                # O texto, a entidade no texto e a classe são repetidas
                text_new.append(text[i])
                entities_new.append(entities[i])
                classes_NER_new.append(classes_NER[i])
            except:
                print(ent)
            
text = np.array(text_new)
entities = np.array(entities_new)
classes_NER = np.array(classes_NER_new) 
ENT_ID = np.array(ENT_ID_new)
URI_Vec = np.array(URI_Vec_new)
            
print('Total de sentenças anotadas: ', n)
print('N° de sentenças: ', len(text))
print ('N° de entidades: ', len(entities))
print('N° de classes: ', len(classes_NER))
print('N° de entidades_ID: ', len(ENT_ID))       
print('N° de vetores de URI: ', len(URI_Vec))       

#gipsita
#gipsita
#gipsita
#grupo_05
#carbonatos
#dry_hole
#grupo_05
#carbonatos
#grupo_05
#grupo_05
#carbonatos
#carbonatos
#carbonatos
#carbonatos
#grupo_05
#gipsita
#grupo_05
#dry_hole
#grupo_05
#grupo_05
#grupo_05
#grupo_05
#grupo_05
#grupo_05
#carbonatos
#carbonatos
#carbonatos
#carbonatos
#grupo_05
#grupo_05
#dry_hole
#dry_hole
Total de sentenças anotadas:  18820
N° de sentenças:  9844
N° de entidades:  9844
N° de classes:  9844
N° de entidades_ID:  9844
N° de vetores de URI:  9844


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [227]:
np.array(ENT_ID[:10])

array(['#LowerCretaceous', '#BASE_CD_BACIA_270', '#Carboniferous',
       '#BASE_CD_BACIA_030', '#CAMP_CD_CAMPO_0760', '#BASE_CD_BACIA_281',
       '#Mesozoic', '#BASE_CD_BACIA_100',
       '#TEFR_CD_TIPO_EST_FISICA_ROCHA_036',
       '#TEFR_CD_TIPO_EST_FISICA_ROCHA_036'], dtype='<U34')

In [117]:
PetroOntoVec['http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2' + '#LowerCretaceous']

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.6703877 ,  0.265335  ,  0.95278156,  0.89755875,  0.2324965 ,
       -0.0926315 , -0.4794778 , -0.19787806, -0.5810278 , -0.31740925,
        0.20098737, -0.36909622, -0.62432104, -0.35961992, -0.5799979 ,
       -1.1427058 , -2.135309  ,  0.8737656 ,  0.73145604, -0.5567418 ,
       -0.9594028 , -1.4623563 ,  0.40491855,  1.8565482 , -0.111449  ,
        0.29665458, -1.8804556 , -0.83291215, -1.0132178 ,  0.15205827,
       -1.0875425 , -0.5876696 ,  0.50690794, -0.35811982, -1.1714823 ,
       -0.78757334, -0.6718937 ,  0.28364375, -0.767629  ,  1.32649   ,
       -1.0079952 ,  0.6938932 , -0.07394186,  0.12490686, -1.1915737 ,
       -0.27666104, -0.06619169, -0.74382967, -1.0040956 , -1.2803016 ,
        0.13112156,  0.95597464,  0.38763887, -0.30659407,  0.12257597,
       -0.3563015 ,  1.300516  , -0.14993899,  0.16064867, -0.51136756,
        1.2480568 ,  0.9804533 , -0.02366812, -0.85186   , -0.38977754,
        0.5635437 ,  1.5994214 ,  0.5695297 ,  0.82635206, -2.05